In [2]:
import subprocess
import json
import avro.schema
import avro.io
import io
import datetime
import uuid
import time
import sys
from io import BytesIO     # for handling byte strings
from io import StringIO    # for handling unicode strings
import avro
from random import randint
#from avro.datafile import DataFileWriter
from avro.io import DatumWriter
from argparse import RawTextHelpFormatter

#https://github.com/pndaproject/example-applications/blob/develop/jupyter-notebooks/notebooks/Example%20Platform-library%20PySpark%20Notebook.ipynb

In [4]:
def generate_sample_datasets (host_ips, metric_ids, year, month, day, hour):
    avro_schema = '''
           {"namespace": "pnda.entity",
             "type": "record",
             "name": "event",
             "fields": [
                {"name": "timestamp", "type": "long"},
                {"name": "source",    "type": "string"},
                {"name": "rawdata",   "type": "bytes"}
            ]}'''
    schema = avro.schema.parse(avro_schema)
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    #create hdfs folder structure
    dir = create_hdfs_dirs (year, month, day, hour)
    filename = str(uuid.uuid4()) + '.avro'
    filepath = dir + filename
    tmp_file = '/tmp/' + filename
    writer = DataFileWriter(open(tmp_file, "w"), DatumWriter(), schema)
    start_dt = datetime.datetime(year, month, day, hour, 0, 0) 
    start_ts = int(time.mktime(start_dt.timetuple()))
    end_dt = start_dt.replace(hour=hour+1)
    end_ts = int(time.mktime(end_dt.timetuple()))

    for ts in xrange(start_ts, end_ts, 1):
        #generate random pnda record on per host ip basis
        for host_ip in host_ips:
            record = {}
            record['timestamp'] = (ts * 1000)
            record['source'] = 'samples'
            record['rawdata'] = generate_random_metrics(host_ip, metric_ids)
           #encode avro
            writer.append(record)
    writer.close()
    #subprocess.Popen(['hadoop', 'fs', '-copyFromLocal', tmp_file, dir])
    return filepath

def generate_random_metrics (host_ip, metric_ids):
    raw_data = {'host': host_ip}
    for id in metric_ids:
        raw_data[id] = str(randint(0, 100))
    return json.dumps(raw_data).encode('utf-8')

def create_hdfs_dirs (year, month, day, hour):
    dir = "datas/source=samples/year=%0d/month=%02d/day=%02d/hour=%02d/" % (year, month, day, hour)
    subprocess.Popen(['hadoop', 'fs', '-mkdir', '-p', dir])
    return dir    

#example host ips (update as you wish)
host_ips = ['10.0.0.1', '10.0.0.2', '10.0.0.3']
#example metric list (update as you wish)
metrics=['in_bytes', 'out_bytes', 'in_pks', 'out_pks']
#generate example datasets (update year, month, day, and hour as you wish)
generate_sample_datasets(host_ips, metrics, 2016, 4, 26, 16)

NameError: name 'basestring' is not defined